In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10, 5)


# Lecture 03 - Messy Data 🤯

## DSC 80, Fall 2022

## Today, in DSC 80...

- The importance of understanding where data comes from.
- How can data be messy?
- Missing data.

## Announcements 📣

- 

## Introduction to messy data

### There is no such thing as "raw data"!

* Data are the result of measurements that must be recorded.
* Humans design the measurements and record the results.
* Data is **always** an imperfect record of the underlying processing being measured.

### Data generating process

* A **data generating process** is the underlying, real-world (probabilistic) mechanism that generates observed data. 
* Observed data is an incomplete artifact of the data generating process.
* **A data generating process is what a statistical model attempts to describe.**
    - From DSC 10: a model is a set of assumptions about how data were generated.
    - More on this later in the quarter.
- Data cleaning requires an understanding of the data generating process.

### Example: COVID case counts 🦠

Suppose our **goal** is to determine the number of COVID cases in the US **yesterday**.
- What are we really asking for – the number of people who tested positive yesterday, or the number of people who contracted COVID yesterday?
- Tested positive on what type of test? How accurate is that type of test?
- How often are test results reported? Is there a delay in when test results are reported?

<center><img src='imgs/christmas.png' width=70%></center>

Why do you think so few cases were reported on Christmas Day – is it because COVID was less prevalent on Christmas Day as compared to the days before and after, or is it likely for some other reason? 🎅

### Data provenance

- As data scientists, we often need to work with datasets that others collected, for a purpose that is different than our current interest.
- As such, it's important to understand the "story" of how a dataset came to be, or the **provenance** of the data. Specifically, we need to be aware of:
    1. Assumptions about the data generating process.
    2. How the initial values in the dataset came to be.  
    3. How any data processing or storage decisions affected the values in the dataset.

The bigger picture question we're asking here is, **can we trust our data?**

### Data cleaning 🧹

- Data cleaning is the process of transforming data so that it best represents the underlying data generating process.

- In practice, data cleaning is often detective work to understand data provenance.
    - **Always be skeptical of your data!**

### Keys to data cleaning

Data cleaning often addresses: 

* The **structure** of the recorded data.
    - Is the data stored in a tabular format (e.g. CSV, SQL, Google Sheets) or in another format (JSON, XML)?
    - Are the individuals properly represented as rows?
* The **encoding** and **format** of the values in the data.
    - Are the data types of all columns reflective of the **kinds of data** they contain?
* Corrupt and "**incorrect**" data, and missing values.
    - Were there flaws in the data recording process? In other words, is our data **faithful** to the data generating process?
    
Let's focus on the latter two.

## Kinds of data

### Kinds of data

<center><img src='imgs/data-types.png' width=90%></center>

### Discussion Question

Determine the kind of each of the following variables.
- Fuel economy in miles per gallon.
- Number of quarters at UCSD.
- Class standing (freshman, sophomore, etc.).
- Income bracket (low, medium, high).
- Bank account number.

### Example: DSC 80 students

In the next cell, we'll load in an example dataset containing information about past DSC 80 students.

- `'PID'` and `'Student Name'`: student PID and name.
- `'Month'`, `'Day'`, `'Year'`: date when the student was accepted to UCSD.
- `'2021 tuition'` and `'2022 tuition'`: amount paid in tuition in 2021 and 2022, respectively.
- `'Percent Growth'`: growth between the two aforementioned columns.
- `'Paid'`: whether or not the student has paid tuition for this quarter yet.
- `'DSC 80 Final Grade'`: either `'Pass'`, `'Fail'`, or a number.

What needs to be changed in the DataFrame to compute statistics?

In [ ]:
students = pd.read_csv(os.path.join('data', 'students.csv'))
students

### How much has each student paid in total tuition in 2021 and 2022?

In [ ]:
students

In [ ]:
total = students['2021 tuition'] + students['2022 tuition']
total

### Check the data types of `students`!

* What kinds of data should each column have?
    - Qualitative or quantitative?
    - Discrete or continuous?
    - Ordinal or nominal?
* What data type *should* each column have?

* Use the `dtypes` attribute (or the `info` method) to peek at the data types.

In [ ]:
students.dtypes

### Cleaning `'2021 tuition'` and `'2022 tuition'`

* `'2021 tuition'` and `'2022 tuition'` are stored as `object`s (strings), not numerical values.
* The `'$'` character causes the entries to be interpreted as strings.
* We can use `str` methods to strip the dollar sign.

In [ ]:
# This won't work. Why?
students['2021 tuition'].astype(float)

In [ ]:
# That's better!
students['2021 tuition'].str.strip('$').astype(float)

We can loop through the columns of `students` to apply the above procedure. (Looping through columns is fine, just avoid looping through rows.)

In [ ]:
for col in students.columns:
    if 'tuition' in col:
        students[col] = students[col].str.strip('$').astype(float)
        
students

Alternatively, we can do this without a loop by using `str.contains` to find only the columns that contain tuition information.

In [ ]:
cols = students.columns.str.contains('tuition')
students.loc[:, cols] = students.loc[:, cols].astype(float)
students

### Cleaning `'Paid'`

* Currently, `'Paid'` contains the strings `'Y'` and `'N'`.
    * `'Y'`s and `'N'`s typically result from manual data entry.
* The `'Paid'` column should contain `True`s and `False`s, or `1`s and `0`s.
* Solutions:
    - Use the `replace` Series method.
    - Create a Boolean Series through comparison.

In [ ]:
students['Paid'].replace({'Y': True, 'N': False})

In [ ]:
students['Paid'].value_counts()

In [ ]:
students['Paid'] = students['Paid'] == 'Y'
students

### Cleaning `'Month'`, `'Day'`, and `'Year'`
* Currently, these are stored separately using the `int64` data type. This could be *fine* for certain purposes, but ideally they are stored as a single column (e.g. for sorting).
* Solutions:
    * Store dates as strings of the form `'YYYY-MM-DD'`.
    * Store dates as `datetime64` objects (later).

In [ ]:
(
    students['Year'].astype(str) + '-' + 
    students['Month'].astype(str).str.zfill(2) + '-' + 
    students['Day'].astype(str).str.zfill(2)
)

Note:
- Due to **broadcasting**, we were able to add a Series to a string.
- The `zfill` string method adds zeroes to the start of a string until it reaches the specified length.

### Cleaning `'DSC 80 Final Grade'`

* Currently, `'DSC 80 Final Grade'`s are stored as `object`s (strings).
* Unless we somehow store this column to a numeric type, we can't do any arithmetic with it.
* However, due to the existence of strings like `'Pass'`, we can't use `astype` to convert it.
* Solution: use `pd.to_numeric(s, errors='coerce')`, where `s` is a Series.
    - ⚠️ Be careful with this!
    - `errors='coerce'` can cause uninformed destruction of data.

In [ ]:
# Won't work!
students['DSC 80 Final Grade'].astype(int)

In [ ]:
pd.to_numeric(students['DSC 80 Final Grade'], errors='coerce')

In [ ]:
students['DSC 80 Final Grade'] = pd.to_numeric(students['DSC 80 Final Grade'], errors='coerce')
students

In [ ]:
pd.to_numeric?

### Cleaning `'Student Name'`
* We want names to be formatted as `'Last Name, First Name'`, a common format.
* One solution: use the Series `apply` method.
    - If `s` is a Series, `s.apply(func)` applies the function `func` to each entry of `s`.

In [ ]:
students['Student Name']

In [ ]:
def transpose_name(name):
    firstname, lastname = name.split()
    return lastname + ', ' + firstname

transpose_name('King Triton')

In [ ]:
students['Student Name'].apply(transpose_name)

### Aside: string methods

`str` methods are useful – use them!
- To use them, access the `str` attribute of Series.
- Then, whatever method/operator comes immediately after will be applied to each element of the Series individually, rather than the Series as a whole.

In [ ]:
parts = students['Student Name'].str.split()
parts

In [ ]:
parts.str[1] + ', ' + parts.str[0]

### More data type ambiguities

- 1649043031 looks like a number, but is probably a date.
    - [Unix timestamps](https://www.unixtimestamp.com) count the number of seconds since January 1st, 1970.

- "USD 1,000,000" looks like a string, but is actually a number **and** a unit.
    
- 92093 looks like a number, but is really a zip code (and isn't equal to 92,093).
    
- Sometimes, `False` appears in a column of country codes. Why might this be? 
🤔

### Example: the Norway problem 🇳🇴

In [ ]:
import yaml

player = '''
name: Magnus Carlsen
age: 31
country: NO
'''

In [ ]:
yaml.safe_load(player)

## Unfaithful data

### Is the data "faithful" to the DGP?

- In other words, how well does the data represent reality?

- Does the data contain unrealistic or "incorrect" values?
    - Dates in the future for events in the past.
    - Locations that don't exist.
    - Negative counts.
    - Misspellings of names.
    - Large outliers.

### Is the data "faithful" to the DGP?
    
- Does the data violate obvious dependencies?
    - Age and birthday don't match. 
- Was the data entered by hand?
     - Spelling errors.
     - Fields shifted.
     - Did the form require fields or provide default values?  
- Are there obvious signs of data falsification (aka "curbstoning")?
    - Repeated names.
    - Fake looking email addresses.
    - Repeated use of uncommon names or fields.

<center><img src='imgs/data-sd.png' width=70%></center>

### Example: Police vehicle stops 🚔

The dataset we're working with contains all of the vehicle stops that the San Diego Police Department made in 2016.

<center><img src="imgs/image_5.png"/></center>

### General questions

1. Check the data types. Notice any issues?
2. Do string fields have consistent values?
3. Are there missing values that we don't understand?
4. Are all values within a reasonable range?
5. How do we deal with the messiness we find?

In [ ]:
stops = pd.read_csv('data/vehicle_stops_2016_datasd.csv')
stops.head()

### Data types
* Are the data types correct?
* If not, are they easily fixable?

In [ ]:
stops.head(1)

In [ ]:
stops.info()

### Unfaithfulness
* Are there suspicious values?
* If a value is suspicious, can we trust the observation?
* For example, consider `'subject_age'` – some are too high to be true, some are too low to be true.

In [ ]:
stops['subject_age'].unique()

In [ ]:
ages = pd.to_numeric(stops['subject_age'], errors='coerce')
ages.describe()

Ages range all over the place, from 0 to 220. Was a 220 year old really pulled over?

In [ ]:
stops.loc[ages > 100]

In [ ]:
ages.loc[(ages >= 0) & (ages < 16)].value_counts()

In [ ]:
stops.loc[(ages >= 0) & (ages < 16)]

### Unfaithful `'subject_age'`

* Ages of `'No Age'` and `0` are likely explicit null values.
* What do we do about the exceptionally small and large ages?
    - Do we throw the entire row away, even if the rest of row is well-formed?
* What about the 14 and 15 year olds?
    - Each has more than one occurrence – these could be real entries!

### Human-entered data
* Which fields were likely entered by a human?
* Which fields were likely generated by code?
    - What was the original source?

Let's look at all unique stop causes. Notice that there are three different causes related to bicycles, which should probably all fall under the same cause.

In [ ]:
stops['stop_cause'].value_counts()

Let's plot the distribution of ages, within a reasonable range (15 to 85). What do you notice?

In [ ]:
# DSC 10 review: what does density=True do?
ages.loc[(ages > 15) & (ages <= 85)].plot(kind='hist', density=True, bins=70, ec='w');

Now let's look at the first few and last few rows of `stops`.

In [ ]:
stops[['timestamp', 'stop_date', 'stop_time']].head()

In [ ]:
stops[['timestamp', 'stop_date', 'stop_time']].tail(10)

Do you think `'-0:81'` is a time that a computer would record?

### Unfaithful data vs. outliers

* Unfaithful data are data that don't accurately represent the data generating process.
* Outliers are "unusual" observations, unlike the rest of the data. They may be real, or they may be unfaithful.
    - For instance, it's possible that a 102-year old was pulled over for speeding.
* The two are hard to tell apart; doing so often requires research and domain knowledge.

### Outliers

* **Consistently "incorrect" values**.
    - Example: Recorded ages of -1 or 99.
    - These are often "default" values, often used when a value is missing.
    - Solution: Change the value to the correct one if it is known!
    
* **Abnormal artifacts from the data collection process**.
    - Example: Spikes in recorded ages at round numbers (25, 30, 35, 40), or spikes in recorded COVID cases on Mondays.
    - Solution: Try "smoothing", e.g. binning the ages.
        
* **Unreasonable outliers**.
    - Example: Age of 200.
    - Solution: Not sure. Could remove the row. Could be indicative of a bug in the data collection process. Could be real!

### Reminder: tools 🛠

You'll use the following methods regularly when initially exploring a dataset.

- `.describe()`: see basic numerical information about a Series/DataFrame.
- `.info()`: see data types and the number of missing values in a Series/DataFrame.
- `.value_counts()`: see the distribution of a categorical variable.
- `.plot(kind='hist')`: plot the distribution of a numerical variable.

In [ ]:
stops = pd.read_csv('data/vehicle_stops_2016_datasd.csv')
stops.head()

## Missing values

### Where'd you go?

* Missing values in a dataset can occur from:
    - Intentional logic, where a value doesn't make sense.
    - A non-response in the measurement process.
    - Mistakes in the data recording process.
    - ...
* Another term for "missing" is "null".
    
* Missing values are most often encoded with `NULL`, `None`, `NaN`, `''`, etc.

### Common representations of "null"

- All forms of `0` (e.g. `0`, `'0'`, `'zero'`) are common substitutes for null.
- -1 is common if a column must be non-negative.
- 1900 and 1970 are common if a non-null date is required.
    - Remember, Unix time starts counting from January 1, 1970.

### Common representations of "null"

- Some common representations for "null" are also real values themselves!
- For instance, the point 0°00'00.0"N+0°00'00.0"E in the South Atlantic Ocean is called "Null Island."

<center><img src='imgs/null.png' width=60%></center>

- [This person's name is Mr. Null!](https://www.wired.com/2015/11/null/)

### Missing values in the stops dataset

What are the non-`np.NaN` null values in the stops dataset?
- Service Area: `'Unknown'`.
- Subject Age: `0`, `'No Age'`.
- Others?

In [ ]:
stops

### Finding null values in `pandas`

* Null values are encoded using NumPy's `NaN` value, which is of type `float`.
* The `isna` method for DataFrame/Series detects missing values.
    - It returns a Boolean DataFrame/Series.
    - `isnull` is equivalent to `isna`.

In [ ]:
type(np.NaN)

In [ ]:
# All of the rows where the subject age is missing
stops[stops['subject_age'].isna()]

In [ ]:
# Proportion of values missing in the subject_age column
stops['subject_age'].isna().mean()

In [ ]:
# Proportion of missing values in all columns
stops.isna().mean()

### Dropping observations with null values
- The `dropna` method:
    - when used on a Series, returns a new Series with all null entries removed.
    - when used on a DataFrame, returns a new DataFrame where all rows with at least one null value are removed.
- Don't drop rows unless absolutely necessary!
    - Usually, there is still useful information in the other columns.

In [ ]:
stops.head()

In [ ]:
stops.dropna().head()

In [ ]:
stops.shape

In [ ]:
stops.dropna().shape

### Dropping observations with null values

When used on a DataFrame:

* `.dropna()` drops **rows** containing **at least one** null value.
* `.dropna(how='all')` drops **rows** containing **only** null values.
* `.dropna(axis=1)` drops **columns** containing at least one null value.
* Other keyword arguments: `thresh`, `subset`.

In [ ]:
nans = pd.DataFrame([[0, 1, np.NaN], [np.NaN, np.NaN, np.NaN], [1, 2, 3]], columns='A B C'.split())
nans

In [ ]:
nans.dropna(how='any')

In [ ]:
nans.dropna(how='all')

In [ ]:
nans.dropna(axis=1)

In [ ]:
nans.dropna(subset=['A', 'B'])

### Filling null values

The `fillna` method replaces all null values. Specifically:

* `.fillna(val)` fills null entries with the value `val`.
* `.fillna(dict)` fills null entries using a dictionary `dict` of column/row values.
* `.fillna(method='bfill')` and `.fillna(method='ffill')` fill null entries using neighboring non-null entries.

In [ ]:
nans

In [ ]:
# Filling all NaNs with the same value
nans.fillna('billy')

In [ ]:
# Filling NaNs differently for each column
nans.fillna({'A': 'f0', 'B': 'f1', 'C': 'f2'})

In [ ]:
# Dictionary of column means
# Note that most numerical methods ignore null values
means = {c: nans[c].mean() for c in nans.columns}
means

In [ ]:
# Filling NaNs with column means
nans.fillna(means)

In [ ]:
# Another way of doing the same thing
nans.apply(lambda x: x.fillna(x.mean()), axis=0)

In [ ]:
nans

In [ ]:
# bfill stands for "backfill"
nans.fillna(method='bfill')

In [ ]:
# ffill stands for "forward fill"
nans.fillna(method='ffill')

### Data types and `np.NaN`

* The result of *any* comparison (`==`, `!=`, `<`, `>`) with `np.NaN` is `False`.
* Instead, use the function `pd.isna`, which returns whether the argument is `np.NaN` or `None`.
    - Can also use `pd.isnull`.
* Remember, `NaN` is of type `float` – watch out for type coercion!

In [ ]:
nans

In [ ]:
np.NaN == np.NaN

In [ ]:
pd.isna(np.NaN)

In [ ]:
nans.isna()

In [ ]:
nans.isnull()

In [ ]:
# Since np.NaN is a float, the Series is of type float despite the two ints
pd.Series([0, 1, np.NaN])

### More soon...

- That's all we'll discuss regarding missing values for now.
- However, once we recap hypothesis and permutation testing, we will introduce the idea of **imputation**, in which we will learn how to fill missing values using other information in the DataFrame.
- Stay tuned!

## Next time, in DSC 80...

... hypothesis testing.